In [1]:
import os
print(os.getcwd())

/home/sagemaker-user/research_methodology_extraction/src/00_system


In [6]:
import pathlib
if '__file__' in globals():
    print(pathlib.Path(__file__).parent.resolve())
else:
    print(pathlib.Path().absolute())

/home/sagemaker-user/research_methodology_extraction/src/00_system


In [2]:
import sys
print(sys.executable)
print(sys.prefix)

/home/sagemaker-user/.conda/envs/python_311/bin/python
/home/sagemaker-user/.conda/envs/python_311


In [3]:
import platform
print(platform.python_version())

3.11.13


In [4]:
import os
import importlib
sagemaker_ai_req_filepath = '/home/ec2-user/SageMaker/research_methodology_extraction/requirements.txt'
sagemaker_studio_req_filepath = '/home/sagemaker-user/research_methodology_extraction/requirements.txt'
if os.path.exists(sagemaker_ai_req_filepath):
    with open(sagemaker_ai_req_filepath) as reqstxt:
        reqscontent = reqstxt.read()
elif os.path.exists(sagemaker_studio_req_filepath):
    with open(sagemaker_studio_req_filepath) as reqstxt:
        reqscontent = reqstxt.read()

reqslines = reqscontent.split('\n')
reqs = {}
for reqsline in reqslines:
    reqsline_parts = reqsline.split('#')
    reqsline_pure = reqsline_parts[0]
    module, required_version = reqsline_pure.strip().split('==')
    version_parts = required_version.split('+')
    required_version = version_parts[0]  # Ignore any local version suffix like +cu124
    reqs[module] = required_version
    locals()[module] = importlib.import_module(module)
    installed_version = locals()[module].__version__
    installed_version_parts = installed_version.split('+')
    installed_version_pure = installed_version_parts[0]  # Ignore any local version suffix like +cu124
    print(f'Installed {module}.__version__=={installed_version_pure}', 'as expected' if installed_version_pure==required_version else f'ERROR: expected {required_version}')

Installed numexpr.__version__==2.11.0 as expected
Installed pandas.__version__==2.3.1 as expected
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Installed sagemaker.__version__==2.250.0 as expected
Installed transformers.__version__==4.49.0 as expected
Installed datasets.__version__==4.0.0 as expected
Installed accelerate.__version__==1.10.0 as expected
Installed torch.__version__==2.5.1 as expected
Installed ipywidgets.__version__==8.1.7 as expected


In [ ]:
# For SageMaker AI only

import json
import boto3
import base64
with open('/etc/opt/ml/sagemaker-notebook-instance-config.json') as json_file:
    notebook_config = json.load(json_file)
print('notebook_config:', notebook_config)
print("notebook_config['notebook_uri']:", notebook_config['notebook_uri'])
# Not using regex on purpose. Parsing an already double-escaped regex pattern is doable, but is usually very brittle. 
# In this case it's more robust to build the pattern matching on simple splitting based on structures we can assume an automatically generated configuration string always has.
parts_first = notebook_config['notebook_uri'].split('(')
print('parts_first:', parts_first)
parts_second = parts_first[1].split('|')
print('parts_second:', parts_second)
notebook_instance_name = parts_second[0]
print(f'notebook_instance_name: {notebook_instance_name}')

boto_sagemaker = boto3.client('sagemaker')
notebok_instance_description = boto_sagemaker.describe_notebook_instance(
    NotebookInstanceName = notebook_instance_name
)
print('notebok_instance_description: ', notebok_instance_description)
notebok_instance_lifecycle_config_name = notebok_instance_description['NotebookInstanceLifecycleConfigName']
print('notebok_instance_lifecycle_config_name:', notebok_instance_lifecycle_config_name)
notebok_instance_lifecycle_config_description = boto_sagemaker.describe_notebook_instance_lifecycle_config(
    NotebookInstanceLifecycleConfigName = notebok_instance_lifecycle_config_name
)
print('notebok_instance_lifecycle_config_description_', notebok_instance_lifecycle_config_description)
lifecycle_on_create_base64 = notebok_instance_lifecycle_config_description['OnCreate'][0]['Content']
lifecycle_on_start_base64 = notebok_instance_lifecycle_config_description['OnStart'][0]['Content']
print('lifecycle_on_create_base64:', lifecycle_on_create_base64)
print('lifecycle_on_start_base64:', lifecycle_on_start_base64)
lifecycle_on_create_string = '\n'.join(f'{base64.b64decode(lifecycle_on_create_base64)}'.split('\\n'))
lifecycle_on_start_string = '\n'.join(f'{base64.b64decode(lifecycle_on_start_base64)}'.split('\\n'))
print('lifecycle_on_create_string:', '\n', lifecycle_on_create_string)
print('lifecycle_on_start_string:', '\n', lifecycle_on_start_string)